# Context Window Prioritization with Contextual Bandits

This notebook demonstrates how to use **Contextual Bandits** for intelligent context window management in LLM-based agentic systems.

## Problem Statement

When working with LLMs, we often have more context available than fits in the context window. We need to intelligently select which pieces of context to include to maximize response quality while staying within token limits.

**Key Challenges:**
- Variable relevance of context items depending on query type
- Limited token budget
- Need to balance exploration (trying different strategies) vs exploitation (using known good strategies)
- Different users may have different preferences

## Why Contextual Bandits?

Contextual bandits are ideal for this problem because:
1. **Low complexity** - Simple to implement and deploy
2. **Fast adaptation** - Learn from each interaction
3. **Context-aware** - Decisions depend on query features
4. **No delayed rewards needed** - Immediate feedback after each selection

## Architecture Overview

```
Query Features → Contextual Bandit → Selection Strategy → Context Assembly → LLM → Reward
      ↑                                                                              │
      └──────────────────────── Feedback Loop ───────────────────────────────────────┘
```

In [ ]:
# Install required packages
%pip install numpy scipy matplotlib seaborn scikit-learn tiktoken --quiet

In [ ]:
import numpy as np
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional, Any
from enum import Enum
from abc import ABC, abstractmethod
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ Imports successful!")
print("=" * 60)
print("Context Window Prioritization with Contextual Bandits")
print("=" * 60)

## 1. Data Models and Enums

Define the core data structures for queries, context items, and prioritization strategies.

In [ ]:
class QueryType(Enum):
    """Types of queries that require different context prioritization"""
    FACTUAL = "factual"           # Need accurate, specific facts
    ANALYTICAL = "analytical"      # Need comprehensive background
    CREATIVE = "creative"          # Need diverse inspiration
    TROUBLESHOOTING = "troubleshooting"  # Need recent, specific docs
    SUMMARIZATION = "summarization"      # Need broad coverage


class ContextSource(Enum):
    """Sources of context information"""
    CONVERSATION_HISTORY = "conversation_history"
    USER_PROFILE = "user_profile"
    RETRIEVED_DOCS = "retrieved_docs"
    SYSTEM_STATE = "system_state"
    EXTERNAL_API = "external_api"
    CACHED_RESULTS = "cached_results"


class PrioritizationStrategy(Enum):
    """Available prioritization strategies (arms of the bandit)"""
    RECENT_FIRST = "recent_first"           # Prioritize most recent context
    RELEVANCE_FIRST = "relevance_first"     # Prioritize semantic similarity
    DIVERSE_COVERAGE = "diverse_coverage"   # Maximize topic diversity
    USER_HISTORY = "user_history"           # Prioritize user-specific info
    HYBRID_BALANCED = "hybrid_balanced"     # Weighted combination
    SOURCE_PRIORITY = "source_priority"     # Prioritize by source type


@dataclass
class ContextItem:
    """A single piece of context that could be included in the prompt"""
    id: str
    content: str
    source: ContextSource
    token_count: int
    timestamp: float  # Unix timestamp, more recent = higher
    relevance_score: float  # Pre-computed semantic similarity to query
    topic_embedding: np.ndarray  # For diversity calculation
    user_specific: bool = False
    
    def __hash__(self):
        return hash(self.id)


@dataclass 
class Query:
    """Represents an incoming user query"""
    text: str
    query_type: QueryType
    complexity: float  # 0-1, higher = more complex
    user_id: str
    embedding: np.ndarray  # Query embedding for similarity
    timestamp: float


@dataclass
class ContextFeatures:
    """Features extracted from the context for the bandit"""
    query_type: QueryType
    query_complexity: float
    num_available_items: int
    avg_relevance: float
    token_budget: int
    user_history_length: int
    source_distribution: Dict[ContextSource, int]
    
    def to_vector(self) -> np.ndarray:
        """Convert features to numerical vector for the bandit"""
        # One-hot encode query type
        query_type_vec = np.zeros(len(QueryType))
        query_type_vec[list(QueryType).index(self.query_type)] = 1
        
        # Source distribution as proportions
        total_sources = sum(self.source_distribution.values()) or 1
        source_vec = np.array([
            self.source_distribution.get(s, 0) / total_sources 
            for s in ContextSource
        ])
        
        # Combine all features
        return np.concatenate([
            query_type_vec,
            [self.query_complexity],
            [self.num_available_items / 100],  # Normalize
            [self.avg_relevance],
            [self.token_budget / 8000],  # Normalize to typical context size
            [min(self.user_history_length / 50, 1.0)],  # Cap and normalize
            source_vec
        ])


print("✅ Data models defined!")
print(f"   - {len(QueryType)} query types")
print(f"   - {len(ContextSource)} context sources")
print(f"   - {len(PrioritizationStrategy)} prioritization strategies (bandit arms)")

## 2. Prioritization Strategy Implementations

Each strategy represents an "arm" of the contextual bandit. Given a set of context items and a token budget, each strategy selects which items to include.

In [ ]:
class ContextPrioritizer:
    """Implements various context prioritization strategies"""
    
    # Priority weights for source-based prioritization by query type
    SOURCE_PRIORITIES = {
        QueryType.FACTUAL: {
            ContextSource.RETRIEVED_DOCS: 1.0,
            ContextSource.EXTERNAL_API: 0.9,
            ContextSource.CACHED_RESULTS: 0.7,
            ContextSource.CONVERSATION_HISTORY: 0.4,
            ContextSource.USER_PROFILE: 0.3,
            ContextSource.SYSTEM_STATE: 0.2,
        },
        QueryType.ANALYTICAL: {
            ContextSource.RETRIEVED_DOCS: 1.0,
            ContextSource.CONVERSATION_HISTORY: 0.8,
            ContextSource.EXTERNAL_API: 0.7,
            ContextSource.USER_PROFILE: 0.5,
            ContextSource.CACHED_RESULTS: 0.4,
            ContextSource.SYSTEM_STATE: 0.3,
        },
        QueryType.CREATIVE: {
            ContextSource.USER_PROFILE: 0.9,
            ContextSource.CONVERSATION_HISTORY: 0.8,
            ContextSource.RETRIEVED_DOCS: 0.6,
            ContextSource.CACHED_RESULTS: 0.4,
            ContextSource.EXTERNAL_API: 0.3,
            ContextSource.SYSTEM_STATE: 0.2,
        },
        QueryType.TROUBLESHOOTING: {
            ContextSource.SYSTEM_STATE: 1.0,
            ContextSource.CONVERSATION_HISTORY: 0.9,
            ContextSource.RETRIEVED_DOCS: 0.8,
            ContextSource.EXTERNAL_API: 0.6,
            ContextSource.CACHED_RESULTS: 0.5,
            ContextSource.USER_PROFILE: 0.3,
        },
        QueryType.SUMMARIZATION: {
            ContextSource.CONVERSATION_HISTORY: 1.0,
            ContextSource.RETRIEVED_DOCS: 0.8,
            ContextSource.USER_PROFILE: 0.5,
            ContextSource.CACHED_RESULTS: 0.4,
            ContextSource.EXTERNAL_API: 0.3,
            ContextSource.SYSTEM_STATE: 0.2,
        },
    }
    
    @staticmethod
    def select_items(
        items: List[ContextItem],
        strategy: PrioritizationStrategy,
        token_budget: int,
        query: Query
    ) -> List[ContextItem]:
        """Select context items using the specified strategy"""
        
        if strategy == PrioritizationStrategy.RECENT_FIRST:
            return ContextPrioritizer._recent_first(items, token_budget)
        elif strategy == PrioritizationStrategy.RELEVANCE_FIRST:
            return ContextPrioritizer._relevance_first(items, token_budget)
        elif strategy == PrioritizationStrategy.DIVERSE_COVERAGE:
            return ContextPrioritizer._diverse_coverage(items, token_budget)
        elif strategy == PrioritizationStrategy.USER_HISTORY:
            return ContextPrioritizer._user_history(items, token_budget)
        elif strategy == PrioritizationStrategy.HYBRID_BALANCED:
            return ContextPrioritizer._hybrid_balanced(items, token_budget, query)
        elif strategy == PrioritizationStrategy.SOURCE_PRIORITY:
            return ContextPrioritizer._source_priority(items, token_budget, query)
        else:
            raise ValueError(f"Unknown strategy: {strategy}")
    
    @staticmethod
    def _greedy_knapsack(items: List[Tuple[float, ContextItem]], budget: int) -> List[ContextItem]:
        """Greedy knapsack selection based on score/token ratio"""
        # Sort by score (descending)
        sorted_items = sorted(items, key=lambda x: x[0], reverse=True)
        
        selected = []
        remaining_budget = budget
        
        for score, item in sorted_items:
            if item.token_count <= remaining_budget:
                selected.append(item)
                remaining_budget -= item.token_count
                
            if remaining_budget <= 0:
                break
                
        return selected
    
    @staticmethod
    def _recent_first(items: List[ContextItem], budget: int) -> List[ContextItem]:
        """Prioritize most recent context items"""
        scored = [(item.timestamp, item) for item in items]
        return ContextPrioritizer._greedy_knapsack(scored, budget)
    
    @staticmethod
    def _relevance_first(items: List[ContextItem], budget: int) -> List[ContextItem]:
        """Prioritize highest semantic relevance"""
        scored = [(item.relevance_score, item) for item in items]
        return ContextPrioritizer._greedy_knapsack(scored, budget)
    
    @staticmethod
    def _diverse_coverage(items: List[ContextItem], budget: int) -> List[ContextItem]:
        """Maximize topic diversity using MMR-like selection"""
        if not items:
            return []
            
        selected = []
        remaining = list(items)
        remaining_budget = budget
        
        # Start with highest relevance item
        first = max(remaining, key=lambda x: x.relevance_score)
        if first.token_count <= remaining_budget:
            selected.append(first)
            remaining.remove(first)
            remaining_budget -= first.token_count
        
        # Iteratively add items that maximize diversity
        lambda_param = 0.5  # Balance relevance vs diversity
        
        while remaining and remaining_budget > 0:
            best_score = -float('inf')
            best_item = None
            
            for item in remaining:
                if item.token_count > remaining_budget:
                    continue
                    
                # Calculate diversity as min distance to selected items
                if selected:
                    min_similarity = min(
                        np.dot(item.topic_embedding, s.topic_embedding) / 
                        (np.linalg.norm(item.topic_embedding) * np.linalg.norm(s.topic_embedding) + 1e-8)
                        for s in selected
                    )
                    diversity = 1 - min_similarity
                else:
                    diversity = 1.0
                
                # MMR score
                score = lambda_param * item.relevance_score + (1 - lambda_param) * diversity
                
                if score > best_score:
                    best_score = score
                    best_item = item
            
            if best_item is None:
                break
                
            selected.append(best_item)
            remaining.remove(best_item)
            remaining_budget -= best_item.token_count
        
        return selected
    
    @staticmethod
    def _user_history(items: List[ContextItem], budget: int) -> List[ContextItem]:
        """Prioritize user-specific context"""
        # Score: user_specific boost + relevance
        scored = [
            (item.relevance_score + (0.5 if item.user_specific else 0), item)
            for item in items
        ]
        return ContextPrioritizer._greedy_knapsack(scored, budget)
    
    @staticmethod
    def _hybrid_balanced(items: List[ContextItem], budget: int, query: Query) -> List[ContextItem]:
        """Balanced combination of recency, relevance, and source priority"""
        max_ts = max(i.timestamp for i in items) if items else 1
        min_ts = min(i.timestamp for i in items) if items else 0
        ts_range = max_ts - min_ts if max_ts != min_ts else 1
        
        source_weights = ContextPrioritizer.SOURCE_PRIORITIES.get(
            query.query_type, 
            {s: 0.5 for s in ContextSource}
        )
        
        scored = []
        for item in items:
            recency = (item.timestamp - min_ts) / ts_range
            relevance = item.relevance_score
            source_score = source_weights.get(item.source, 0.5)
            
            # Weighted combination
            score = 0.4 * relevance + 0.3 * recency + 0.3 * source_score
            scored.append((score, item))
        
        return ContextPrioritizer._greedy_knapsack(scored, budget)
    
    @staticmethod
    def _source_priority(items: List[ContextItem], budget: int, query: Query) -> List[ContextItem]:
        """Prioritize by source type based on query type"""
        source_weights = ContextPrioritizer.SOURCE_PRIORITIES.get(
            query.query_type,
            {s: 0.5 for s in ContextSource}
        )
        
        scored = [
            (source_weights.get(item.source, 0.5) * item.relevance_score, item)
            for item in items
        ]
        return ContextPrioritizer._greedy_knapsack(scored, budget)


print("✅ Prioritization strategies implemented!")
print("   Strategies:")
for strategy in PrioritizationStrategy:
    print(f"   - {strategy.value}")

## 3. Simulated Environment

Create a simulated environment that:
1. Generates realistic queries and context items
2. Simulates LLM response quality based on context selection
3. Provides rewards based on response quality and token efficiency

In [ ]:
class ContextWindowSimulator:
    """Simulates the context prioritization environment"""
    
    # Ground truth: which strategies work best for each query type
    # This simulates what we'd learn from real user feedback
    OPTIMAL_STRATEGIES = {
        QueryType.FACTUAL: {
            PrioritizationStrategy.RELEVANCE_FIRST: 0.9,
            PrioritizationStrategy.SOURCE_PRIORITY: 0.85,
            PrioritizationStrategy.HYBRID_BALANCED: 0.75,
            PrioritizationStrategy.RECENT_FIRST: 0.5,
            PrioritizationStrategy.DIVERSE_COVERAGE: 0.6,
            PrioritizationStrategy.USER_HISTORY: 0.55,
        },
        QueryType.ANALYTICAL: {
            PrioritizationStrategy.DIVERSE_COVERAGE: 0.9,
            PrioritizationStrategy.HYBRID_BALANCED: 0.85,
            PrioritizationStrategy.RELEVANCE_FIRST: 0.7,
            PrioritizationStrategy.SOURCE_PRIORITY: 0.65,
            PrioritizationStrategy.RECENT_FIRST: 0.5,
            PrioritizationStrategy.USER_HISTORY: 0.6,
        },
        QueryType.CREATIVE: {
            PrioritizationStrategy.USER_HISTORY: 0.9,
            PrioritizationStrategy.DIVERSE_COVERAGE: 0.85,
            PrioritizationStrategy.HYBRID_BALANCED: 0.7,
            PrioritizationStrategy.RELEVANCE_FIRST: 0.5,
            PrioritizationStrategy.RECENT_FIRST: 0.55,
            PrioritizationStrategy.SOURCE_PRIORITY: 0.45,
        },
        QueryType.TROUBLESHOOTING: {
            PrioritizationStrategy.RECENT_FIRST: 0.9,
            PrioritizationStrategy.SOURCE_PRIORITY: 0.85,
            PrioritizationStrategy.HYBRID_BALANCED: 0.8,
            PrioritizationStrategy.RELEVANCE_FIRST: 0.65,
            PrioritizationStrategy.DIVERSE_COVERAGE: 0.5,
            PrioritizationStrategy.USER_HISTORY: 0.7,
        },
        QueryType.SUMMARIZATION: {
            PrioritizationStrategy.HYBRID_BALANCED: 0.9,
            PrioritizationStrategy.DIVERSE_COVERAGE: 0.85,
            PrioritizationStrategy.RECENT_FIRST: 0.75,
            PrioritizationStrategy.RELEVANCE_FIRST: 0.7,
            PrioritizationStrategy.USER_HISTORY: 0.6,
            PrioritizationStrategy.SOURCE_PRIORITY: 0.65,
        },
    }
    
    def __init__(self, num_users: int = 10, seed: int = 42):
        np.random.seed(seed)
        random.seed(seed)
        self.num_users = num_users
        self.users = self._create_users()
        self.interaction_count = 0
        
    def _create_users(self) -> Dict[str, Dict]:
        """Create simulated user profiles with preferences"""
        users = {}
        for i in range(self.num_users):
            user_id = f"user_{i}"
            # Each user has slightly different preferences
            preference_noise = {
                strategy: np.random.normal(0, 0.1)
                for strategy in PrioritizationStrategy
            }
            users[user_id] = {
                "id": user_id,
                "history_length": np.random.randint(5, 100),
                "preference_noise": preference_noise,
                "favorite_query_type": random.choice(list(QueryType)),
            }
        return users
    
    def generate_query(self, user_id: Optional[str] = None) -> Query:
        """Generate a random query"""
        if user_id is None:
            user_id = random.choice(list(self.users.keys()))
            
        query_type = random.choice(list(QueryType))
        complexity = np.random.beta(2, 5)  # Skewed toward simpler queries
        
        return Query(
            text=f"Sample {query_type.value} query",
            query_type=query_type,
            complexity=complexity,
            user_id=user_id,
            embedding=np.random.randn(64),  # Simulated embedding
            timestamp=1000000 + self.interaction_count
        )
    
    def generate_context_items(self, query: Query, num_items: int = 20) -> List[ContextItem]:
        """Generate simulated context items for a query"""
        items = []
        base_timestamp = query.timestamp - 1000
        
        for i in range(num_items):
            source = random.choice(list(ContextSource))
            
            # Relevance depends on source and query type
            base_relevance = np.random.beta(2, 3)
            source_bonus = 0.2 if source in [
                ContextSource.RETRIEVED_DOCS, 
                ContextSource.CONVERSATION_HISTORY
            ] else 0
            relevance = min(1.0, base_relevance + source_bonus + np.random.normal(0, 0.1))
            
            items.append(ContextItem(
                id=f"ctx_{i}",
                content=f"Context item {i} from {source.value}",
                source=source,
                token_count=np.random.randint(50, 500),
                timestamp=base_timestamp + i * 10 + np.random.randint(0, 5),
                relevance_score=max(0, min(1, relevance)),
                topic_embedding=np.random.randn(32),
                user_specific=(source == ContextSource.USER_PROFILE or 
                              (source == ContextSource.CONVERSATION_HISTORY and random.random() > 0.5))
            ))
        
        return items
    
    def get_reward(
        self, 
        query: Query, 
        strategy: PrioritizationStrategy,
        selected_items: List[ContextItem],
        token_budget: int
    ) -> Tuple[float, Dict[str, float]]:
        """
        Calculate reward based on strategy effectiveness.
        Returns (reward, details_dict)
        """
        self.interaction_count += 1
        
        # Base quality from strategy-query match
        base_quality = self.OPTIMAL_STRATEGIES[query.query_type][strategy]
        
        # User-specific adjustment
        user = self.users[query.user_id]
        user_adjustment = user["preference_noise"][strategy]
        
        # Token efficiency bonus/penalty
        tokens_used = sum(item.token_count for item in selected_items)
        efficiency = 1.0 - (tokens_used / token_budget) * 0.1  # Small penalty for using more tokens
        
        # Coverage bonus (did we include relevant items?)
        if selected_items:
            avg_relevance = np.mean([item.relevance_score for item in selected_items])
        else:
            avg_relevance = 0
        
        # Complexity adjustment (harder queries benefit more from good strategies)
        complexity_factor = 1.0 + query.complexity * 0.2
        
        # Final quality with noise
        quality = (base_quality + user_adjustment) * efficiency * complexity_factor
        quality = quality * (0.8 + 0.4 * avg_relevance)  # Relevance matters
        quality += np.random.normal(0, 0.05)  # Add noise
        quality = max(0, min(1, quality))
        
        # Reward combines quality and efficiency
        reward = 0.8 * quality + 0.2 * efficiency
        
        details = {
            "quality": quality,
            "efficiency": efficiency,
            "tokens_used": tokens_used,
            "avg_relevance": avg_relevance,
            "base_strategy_quality": base_quality,
        }
        
        return reward, details


# Test the simulator
simulator = ContextWindowSimulator()
test_query = simulator.generate_query()
test_items = simulator.generate_context_items(test_query)

print("✅ Simulator created!")
print(f"\n📋 Sample query:")
print(f"   Type: {test_query.query_type.value}")
print(f"   Complexity: {test_query.complexity:.2f}")
print(f"   User: {test_query.user_id}")
print(f"\n📦 Generated {len(test_items)} context items:")
print(f"   Total tokens: {sum(i.token_count for i in test_items)}")
print(f"   Sources: {set(i.source.value for i in test_items)}")

## 4. Contextual Bandit Implementations

We implement three contextual bandit algorithms:
1. **Epsilon-Greedy with Linear Model** - Simple but effective
2. **Thompson Sampling with Bayesian Linear Regression** - Better exploration
3. **Upper Confidence Bound (LinUCB)** - Optimism under uncertainty

In [ ]:
class ContextualBandit(ABC):
    """Abstract base class for contextual bandits"""
    
    def __init__(self, n_arms: int, context_dim: int):
        self.n_arms = n_arms
        self.context_dim = context_dim
        self.arms = list(PrioritizationStrategy)
        self.history: List[Dict] = []
        
    @abstractmethod
    def select_arm(self, context: np.ndarray) -> int:
        """Select an arm given the context"""
        pass
    
    @abstractmethod
    def update(self, context: np.ndarray, arm: int, reward: float):
        """Update the model with observed reward"""
        pass
    
    def get_strategy(self, arm_index: int) -> PrioritizationStrategy:
        """Convert arm index to strategy"""
        return self.arms[arm_index]


class EpsilonGreedyLinear(ContextualBandit):
    """
    Epsilon-greedy contextual bandit with linear reward model.
    Uses ridge regression to estimate expected rewards.
    """
    
    def __init__(self, n_arms: int, context_dim: int, epsilon: float = 0.1, 
                 lambda_reg: float = 1.0, decay_rate: float = 0.995):
        super().__init__(n_arms, context_dim)
        self.epsilon = epsilon
        self.initial_epsilon = epsilon
        self.decay_rate = decay_rate
        self.lambda_reg = lambda_reg
        
        # Initialize linear models for each arm
        # A[a] = X'X + lambda*I, b[a] = X'y
        self.A = [np.eye(context_dim) * lambda_reg for _ in range(n_arms)]
        self.b = [np.zeros(context_dim) for _ in range(n_arms)]
        self.theta = [np.zeros(context_dim) for _ in range(n_arms)]  # Weights
        
        self.arm_counts = np.zeros(n_arms)
        self.total_reward = np.zeros(n_arms)
        
    def select_arm(self, context: np.ndarray) -> int:
        """Epsilon-greedy selection with linear predictions"""
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_arms)
        
        # Predict expected reward for each arm
        expected_rewards = [
            np.dot(self.theta[a], context) for a in range(self.n_arms)
        ]
        return int(np.argmax(expected_rewards))
    
    def update(self, context: np.ndarray, arm: int, reward: float):
        """Update the linear model for the selected arm"""
        # Update sufficient statistics
        self.A[arm] += np.outer(context, context)
        self.b[arm] += reward * context
        
        # Recompute weights (ridge regression solution)
        try:
            self.theta[arm] = np.linalg.solve(self.A[arm], self.b[arm])
        except np.linalg.LinAlgError:
            self.theta[arm] = np.linalg.lstsq(self.A[arm], self.b[arm], rcond=None)[0]
        
        self.arm_counts[arm] += 1
        self.total_reward[arm] += reward
        
        # Decay epsilon
        self.epsilon = max(0.01, self.epsilon * self.decay_rate)
        
        self.history.append({
            "arm": arm,
            "reward": reward,
            "epsilon": self.epsilon,
        })


class ThompsonSamplingLinear(ContextualBandit):
    """
    Thompson Sampling with Bayesian linear regression.
    Maintains posterior distribution over weights.
    """
    
    def __init__(self, n_arms: int, context_dim: int, 
                 lambda_reg: float = 1.0, sigma: float = 0.5):
        super().__init__(n_arms, context_dim)
        self.lambda_reg = lambda_reg
        self.sigma = sigma  # Observation noise
        
        # Prior: theta ~ N(0, lambda^-1 * I)
        # Posterior after observing data: theta ~ N(mu, Sigma)
        self.B = [np.eye(context_dim) * lambda_reg for _ in range(n_arms)]  # Precision
        self.mu = [np.zeros(context_dim) for _ in range(n_arms)]  # Mean
        self.f = [np.zeros(context_dim) for _ in range(n_arms)]  # X'y accumulator
        
        self.arm_counts = np.zeros(n_arms)
        
    def select_arm(self, context: np.ndarray) -> int:
        """Thompson sampling: sample from posterior and select best arm"""
        sampled_rewards = []
        
        for a in range(self.n_arms):
            # Sample weights from posterior
            try:
                cov = np.linalg.inv(self.B[a]) * (self.sigma ** 2)
                # Ensure covariance is positive definite
                cov = (cov + cov.T) / 2
                eigvals = np.linalg.eigvalsh(cov)
                if np.min(eigvals) < 0:
                    cov += np.eye(self.context_dim) * (abs(np.min(eigvals)) + 1e-6)
                theta_sample = np.random.multivariate_normal(self.mu[a], cov)
            except np.linalg.LinAlgError:
                theta_sample = self.mu[a] + np.random.randn(self.context_dim) * 0.1
            
            # Predict reward with sampled weights
            expected_reward = np.dot(theta_sample, context)
            sampled_rewards.append(expected_reward)
        
        return int(np.argmax(sampled_rewards))
    
    def update(self, context: np.ndarray, arm: int, reward: float):
        """Update posterior for the selected arm"""
        # Update precision matrix
        self.B[arm] += np.outer(context, context) / (self.sigma ** 2)
        
        # Update reward accumulator
        self.f[arm] += reward * context / (self.sigma ** 2)
        
        # Update posterior mean
        try:
            self.mu[arm] = np.linalg.solve(self.B[arm], self.f[arm])
        except np.linalg.LinAlgError:
            self.mu[arm] = np.linalg.lstsq(self.B[arm], self.f[arm], rcond=None)[0]
        
        self.arm_counts[arm] += 1
        
        self.history.append({
            "arm": arm,
            "reward": reward,
        })


class LinUCB(ContextualBandit):
    """
    Linear Upper Confidence Bound algorithm.
    Balances exploration and exploitation using confidence bounds.
    """
    
    def __init__(self, n_arms: int, context_dim: int, 
                 alpha: float = 1.0, lambda_reg: float = 1.0):
        super().__init__(n_arms, context_dim)
        self.alpha = alpha  # Exploration parameter
        self.lambda_reg = lambda_reg
        
        # Initialize for each arm
        self.A = [np.eye(context_dim) * lambda_reg for _ in range(n_arms)]
        self.b = [np.zeros(context_dim) for _ in range(n_arms)]
        self.theta = [np.zeros(context_dim) for _ in range(n_arms)]
        
        self.arm_counts = np.zeros(n_arms)
        
    def select_arm(self, context: np.ndarray) -> int:
        """Select arm with highest UCB"""
        ucb_values = []
        
        for a in range(self.n_arms):
            # Compute inverse of A
            try:
                A_inv = np.linalg.inv(self.A[a])
            except np.linalg.LinAlgError:
                A_inv = np.linalg.pinv(self.A[a])
            
            # Expected reward
            expected = np.dot(self.theta[a], context)
            
            # Confidence bonus
            confidence = self.alpha * np.sqrt(
                np.dot(context, np.dot(A_inv, context))
            )
            
            ucb = expected + confidence
            ucb_values.append(ucb)
        
        return int(np.argmax(ucb_values))
    
    def update(self, context: np.ndarray, arm: int, reward: float):
        """Update the model for the selected arm"""
        self.A[arm] += np.outer(context, context)
        self.b[arm] += reward * context
        
        try:
            self.theta[arm] = np.linalg.solve(self.A[arm], self.b[arm])
        except np.linalg.LinAlgError:
            self.theta[arm] = np.linalg.lstsq(self.A[arm], self.b[arm], rcond=None)[0]
        
        self.arm_counts[arm] += 1
        
        self.history.append({
            "arm": arm,
            "reward": reward,
        })


print("✅ Contextual bandit algorithms implemented!")
print("   - EpsilonGreedyLinear: Simple epsilon-greedy with linear model")
print("   - ThompsonSamplingLinear: Bayesian approach with uncertainty sampling")
print("   - LinUCB: Optimistic exploration using confidence bounds")

## 5. Context Prioritization System

The main system that orchestrates the bandit, context selection, and learning loop.

In [ ]:
class ContextPrioritizationSystem:
    """
    Main system that uses contextual bandits for intelligent context prioritization.
    """
    
    def __init__(
        self, 
        bandit: ContextualBandit,
        token_budget: int = 4000,
        simulator: Optional[ContextWindowSimulator] = None
    ):
        self.bandit = bandit
        self.token_budget = token_budget
        self.simulator = simulator or ContextWindowSimulator()
        
        self.interaction_history: List[Dict] = []
        self.cumulative_reward = 0
        self.optimal_cumulative = 0  # For regret calculation
        
    def extract_features(
        self, 
        query: Query, 
        items: List[ContextItem]
    ) -> ContextFeatures:
        """Extract features from query and available context"""
        source_dist = defaultdict(int)
        for item in items:
            source_dist[item.source] += 1
            
        user = self.simulator.users.get(query.user_id, {})
        
        return ContextFeatures(
            query_type=query.query_type,
            query_complexity=query.complexity,
            num_available_items=len(items),
            avg_relevance=np.mean([i.relevance_score for i in items]) if items else 0,
            token_budget=self.token_budget,
            user_history_length=user.get("history_length", 0),
            source_distribution=dict(source_dist)
        )
    
    def process_query(
        self, 
        query: Optional[Query] = None,
        items: Optional[List[ContextItem]] = None,
        return_details: bool = False
    ) -> Dict[str, Any]:
        """
        Process a query using the contextual bandit to select prioritization strategy.
        """
        # Generate query and items if not provided
        if query is None:
            query = self.simulator.generate_query()
        if items is None:
            items = self.simulator.generate_context_items(query)
            
        # Extract features and convert to vector
        features = self.extract_features(query, items)
        context_vector = features.to_vector()
        
        # Select strategy using bandit
        arm_index = self.bandit.select_arm(context_vector)
        strategy = self.bandit.get_strategy(arm_index)
        
        # Apply the selected strategy
        selected_items = ContextPrioritizer.select_items(
            items, strategy, self.token_budget, query
        )
        
        # Get reward from simulator
        reward, reward_details = self.simulator.get_reward(
            query, strategy, selected_items, self.token_budget
        )
        
        # Update bandit
        self.bandit.update(context_vector, arm_index, reward)
        
        # Track metrics
        self.cumulative_reward += reward
        
        # Calculate optimal reward for regret
        optimal_strategy = max(
            PrioritizationStrategy,
            key=lambda s: self.simulator.OPTIMAL_STRATEGIES[query.query_type][s]
        )
        optimal_items = ContextPrioritizer.select_items(
            items, optimal_strategy, self.token_budget, query
        )
        optimal_reward, _ = self.simulator.get_reward(
            query, optimal_strategy, optimal_items, self.token_budget
        )
        self.optimal_cumulative += optimal_reward
        
        # Record interaction
        interaction = {
            "query_type": query.query_type.value,
            "strategy": strategy.value,
            "reward": reward,
            "optimal_reward": optimal_reward,
            "regret": optimal_reward - reward,
            "tokens_used": sum(i.token_count for i in selected_items),
            "items_selected": len(selected_items),
            "cumulative_reward": self.cumulative_reward,
            "cumulative_regret": self.optimal_cumulative - self.cumulative_reward,
        }
        self.interaction_history.append(interaction)
        
        if return_details:
            interaction["features"] = features
            interaction["reward_details"] = reward_details
            interaction["selected_items"] = selected_items
            
        return interaction
    
    def run_simulation(self, n_iterations: int, verbose: bool = True) -> List[Dict]:
        """Run multiple iterations of the simulation"""
        if verbose:
            print(f"Running {n_iterations} iterations...")
            
        for i in range(n_iterations):
            self.process_query()
            
            if verbose and (i + 1) % 200 == 0:
                recent_rewards = [h["reward"] for h in self.interaction_history[-200:]]
                avg_reward = np.mean(recent_rewards)
                cumulative_regret = self.interaction_history[-1]["cumulative_regret"]
                print(f"  Iteration {i+1}: Avg Reward (last 200) = {avg_reward:.3f}, "
                      f"Cumulative Regret = {cumulative_regret:.2f}")
                
        return self.interaction_history
    
    def get_learned_policy(self) -> Dict[str, str]:
        """Extract the learned policy (best strategy for each query type)"""
        policy = {}
        
        for query_type in QueryType:
            # Create a synthetic context for this query type
            features = ContextFeatures(
                query_type=query_type,
                query_complexity=0.5,
                num_available_items=20,
                avg_relevance=0.5,
                token_budget=self.token_budget,
                user_history_length=25,
                source_distribution={s: 3 for s in ContextSource}
            )
            context_vector = features.to_vector()
            
            # Get predictions for all arms
            if hasattr(self.bandit, 'theta'):
                predictions = [
                    np.dot(self.bandit.theta[a], context_vector)
                    for a in range(self.bandit.n_arms)
                ]
            elif hasattr(self.bandit, 'mu'):
                predictions = [
                    np.dot(self.bandit.mu[a], context_vector)
                    for a in range(self.bandit.n_arms)
                ]
            else:
                predictions = [0] * self.bandit.n_arms
                
            best_arm = int(np.argmax(predictions))
            best_strategy = self.bandit.get_strategy(best_arm)
            
            policy[query_type.value] = {
                "strategy": best_strategy.value,
                "confidence": predictions[best_arm] if predictions else 0
            }
            
        return policy


# Quick test
n_arms = len(PrioritizationStrategy)
context_dim = len(ContextFeatures(
    QueryType.FACTUAL, 0.5, 20, 0.5, 4000, 25, 
    {s: 3 for s in ContextSource}
).to_vector())

print(f"✅ Context Prioritization System defined!")
print(f"   Arms (strategies): {n_arms}")
print(f"   Context dimension: {context_dim}")

## 6. Training and Algorithm Comparison

Train all three contextual bandit algorithms and compare their performance.

In [ ]:
def train_and_compare(n_iterations: int = 1000, seed: int = 42):
    """Train all bandit algorithms and compare performance"""
    
    # Reset random state for fair comparison
    np.random.seed(seed)
    random.seed(seed)
    
    n_arms = len(PrioritizationStrategy)
    context_dim = len(ContextFeatures(
        QueryType.FACTUAL, 0.5, 20, 0.5, 4000, 25,
        {s: 3 for s in ContextSource}
    ).to_vector())
    
    # Create bandits
    bandits = {
        "Epsilon-Greedy": EpsilonGreedyLinear(n_arms, context_dim, epsilon=0.2),
        "Thompson Sampling": ThompsonSamplingLinear(n_arms, context_dim),
        "LinUCB": LinUCB(n_arms, context_dim, alpha=0.5),
    }
    
    # Also add random baseline
    class RandomBandit(ContextualBandit):
        def select_arm(self, context): return np.random.randint(self.n_arms)
        def update(self, context, arm, reward): pass
    
    bandits["Random"] = RandomBandit(n_arms, context_dim)
    
    results = {}
    
    print("=" * 70)
    print("TRAINING CONTEXTUAL BANDITS FOR CONTEXT PRIORITIZATION")
    print("=" * 70)
    
    for name, bandit in bandits.items():
        print(f"\n{'─' * 50}")
        print(f"Training: {name}")
        print(f"{'─' * 50}")
        
        # Create fresh simulator for each bandit
        np.random.seed(seed)
        random.seed(seed)
        simulator = ContextWindowSimulator(seed=seed)
        
        system = ContextPrioritizationSystem(
            bandit=bandit,
            token_budget=4000,
            simulator=simulator
        )
        
        history = system.run_simulation(n_iterations, verbose=True)
        
        results[name] = {
            "history": history,
            "system": system,
            "final_reward": np.mean([h["reward"] for h in history[-100:]]),
            "final_regret": history[-1]["cumulative_regret"],
            "learned_policy": system.get_learned_policy() if name != "Random" else None
        }
        
    return results


# Run training
results = train_and_compare(n_iterations=1000)

## 7. Visualization and Analysis

Visualize the learning curves, cumulative regret, and learned policies.

In [ ]:
def moving_average(data, window=50):
    """Compute moving average"""
    return np.convolve(data, np.ones(window)/window, mode='valid')


def plot_results(results):
    """Create comprehensive visualization of results"""
    
    fig = plt.figure(figsize=(16, 12))
    colors = {'Epsilon-Greedy': '#e74c3c', 'Thompson Sampling': '#3498db', 
              'LinUCB': '#2ecc71', 'Random': '#95a5a6'}
    
    # 1. Cumulative Reward
    ax1 = fig.add_subplot(2, 2, 1)
    for name, data in results.items():
        rewards = [h["cumulative_reward"] for h in data["history"]]
        ax1.plot(rewards, label=name, color=colors[name], linewidth=2)
    ax1.set_xlabel('Iteration', fontsize=12)
    ax1.set_ylabel('Cumulative Reward', fontsize=12)
    ax1.set_title('Cumulative Reward Over Time', fontsize=14, fontweight='bold')
    ax1.legend(loc='lower right')
    ax1.grid(True, alpha=0.3)
    
    # 2. Cumulative Regret
    ax2 = fig.add_subplot(2, 2, 2)
    for name, data in results.items():
        regrets = [h["cumulative_regret"] for h in data["history"]]
        ax2.plot(regrets, label=name, color=colors[name], linewidth=2)
    ax2.set_xlabel('Iteration', fontsize=12)
    ax2.set_ylabel('Cumulative Regret', fontsize=12)
    ax2.set_title('Cumulative Regret Over Time', fontsize=14, fontweight='bold')
    ax2.legend(loc='upper left')
    ax2.grid(True, alpha=0.3)
    
    # 3. Moving Average Reward
    ax3 = fig.add_subplot(2, 2, 3)
    for name, data in results.items():
        rewards = [h["reward"] for h in data["history"]]
        ma = moving_average(rewards, window=50)
        ax3.plot(ma, label=name, color=colors[name], linewidth=2)
    ax3.set_xlabel('Iteration', fontsize=12)
    ax3.set_ylabel('Reward (50-iter MA)', fontsize=12)
    ax3.set_title('Average Reward (Smoothed)', fontsize=14, fontweight='bold')
    ax3.legend(loc='lower right')
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0.85, color='black', linestyle='--', alpha=0.5, label='Optimal ~0.85')
    
    # 4. Final Performance Comparison
    ax4 = fig.add_subplot(2, 2, 4)
    names = list(results.keys())
    final_rewards = [results[n]["final_reward"] for n in names]
    bar_colors = [colors[n] for n in names]
    bars = ax4.bar(names, final_rewards, color=bar_colors, edgecolor='black', linewidth=1.5)
    ax4.set_ylabel('Final Avg Reward (last 100)', fontsize=12)
    ax4.set_title('Final Performance Comparison', fontsize=14, fontweight='bold')
    ax4.set_ylim(0, 1)
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, val in zip(bars, final_rewards):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{val:.3f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return fig


# Plot the results
fig = plot_results(results)

In [ ]:
# Print learned policies and compare to ground truth
print("\n" + "=" * 70)
print("LEARNED POLICIES vs GROUND TRUTH")
print("=" * 70)

# Ground truth optimal strategies
ground_truth = {
    query_type.value: max(
        strategies.items(), 
        key=lambda x: x[1]
    )[0].value
    for query_type, strategies in ContextWindowSimulator.OPTIMAL_STRATEGIES.items()
}

print("\n📊 Ground Truth Optimal Strategies:")
print("-" * 40)
for qt, strategy in ground_truth.items():
    print(f"  {qt:18s} → {strategy}")

for algo_name in ["Epsilon-Greedy", "Thompson Sampling", "LinUCB"]:
    policy = results[algo_name]["learned_policy"]
    print(f"\n🤖 {algo_name} Learned Policy:")
    print("-" * 40)
    correct = 0
    for qt, info in policy.items():
        learned = info["strategy"]
        is_correct = learned == ground_truth[qt]
        correct += is_correct
        symbol = "✅" if is_correct else "❌"
        print(f"  {qt:18s} → {learned:20s} {symbol}")
    print(f"  Accuracy: {correct}/{len(policy)} ({100*correct/len(policy):.0f}%)")

In [ ]:
# Visualize strategy selection distribution by query type
def plot_strategy_distribution(results, algo_name="Thompson Sampling"):
    """Plot heatmap of strategy selections by query type"""
    
    history = results[algo_name]["history"]
    
    # Count strategy selections per query type
    query_types = [qt.value for qt in QueryType]
    strategies = [s.value for s in PrioritizationStrategy]
    
    counts = np.zeros((len(query_types), len(strategies)))
    
    for h in history:
        qt_idx = query_types.index(h["query_type"])
        st_idx = strategies.index(h["strategy"])
        counts[qt_idx, st_idx] += 1
    
    # Normalize to percentages
    row_sums = counts.sum(axis=1, keepdims=True)
    percentages = np.where(row_sums > 0, counts / row_sums * 100, 0)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    im = ax.imshow(percentages, cmap='YlGnBu', aspect='auto')
    
    # Add colorbar
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel('Selection %', rotation=-90, va="bottom", fontsize=12)
    
    # Set ticks
    ax.set_xticks(np.arange(len(strategies)))
    ax.set_yticks(np.arange(len(query_types)))
    ax.set_xticklabels([s.replace('_', '\n') for s in strategies], fontsize=10)
    ax.set_yticklabels(query_types, fontsize=11)
    
    # Rotate x labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    # Add percentage labels
    for i in range(len(query_types)):
        for j in range(len(strategies)):
            val = percentages[i, j]
            color = "white" if val > 50 else "black"
            ax.text(j, i, f'{val:.0f}%', ha="center", va="center", 
                   color=color, fontsize=9, fontweight='bold')
    
    ax.set_title(f'Strategy Selection Distribution by Query Type ({algo_name})', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('Prioritization Strategy', fontsize=12)
    ax.set_ylabel('Query Type', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    return fig


# Plot strategy distribution for best performing algorithm
fig = plot_strategy_distribution(results, "Thompson Sampling")

## 8. Production Integration Example

A production-ready class that can be integrated into real agentic systems.

In [ ]:
import json
from datetime import datetime
import pickle


class ProductionContextOptimizer:
    """
    Production-ready context window optimizer using contextual bandits.
    
    Features:
    - Multiple algorithm support (Thompson Sampling recommended)
    - Persistence (save/load learned models)
    - Monitoring and logging
    - Fallback strategies for cold start
    - A/B testing support
    """
    
    def __init__(
        self,
        algorithm: str = "thompson_sampling",
        token_budget: int = 4000,
        exploration_rate: float = 0.1,
        model_path: Optional[str] = None
    ):
        self.token_budget = token_budget
        self.algorithm = algorithm
        
        # Initialize bandit
        n_arms = len(PrioritizationStrategy)
        # Feature dim: query_type(5) + complexity(1) + num_items(1) + avg_relevance(1) 
        #              + budget(1) + history(1) + sources(6) = 16
        self.context_dim = 16
        
        if algorithm == "thompson_sampling":
            self.bandit = ThompsonSamplingLinear(n_arms, self.context_dim)
        elif algorithm == "linucb":
            self.bandit = LinUCB(n_arms, self.context_dim, alpha=0.5)
        elif algorithm == "epsilon_greedy":
            self.bandit = EpsilonGreedyLinear(n_arms, self.context_dim, epsilon=exploration_rate)
        else:
            raise ValueError(f"Unknown algorithm: {algorithm}")
        
        # Load existing model if provided
        if model_path:
            self.load_model(model_path)
        
        # Monitoring
        self.request_count = 0
        self.total_reward = 0
        self.strategy_counts = defaultdict(int)
        self.recent_rewards = []
        
    def prioritize_context(
        self,
        query_text: str,
        query_type: str,
        query_complexity: float,
        available_items: List[Dict],
        user_history_length: int = 0
    ) -> Tuple[List[Dict], str, Dict]:
        """
        Select and prioritize context items for an LLM call.
        
        Args:
            query_text: The user's query
            query_type: One of: factual, analytical, creative, troubleshooting, summarization
            query_complexity: 0-1, estimated complexity
            available_items: List of context items with keys: 
                            id, content, source, token_count, timestamp, relevance_score
            user_history_length: Length of user's conversation history
            
        Returns:
            (selected_items, strategy_used, metadata)
        """
        self.request_count += 1
        
        # Convert to internal format
        context_items = self._convert_items(available_items)
        query_type_enum = QueryType(query_type)
        
        # Build context features
        source_dist = defaultdict(int)
        for item in context_items:
            source_dist[item.source] += 1
            
        features = ContextFeatures(
            query_type=query_type_enum,
            query_complexity=query_complexity,
            num_available_items=len(context_items),
            avg_relevance=np.mean([i.relevance_score for i in context_items]) if context_items else 0,
            token_budget=self.token_budget,
            user_history_length=user_history_length,
            source_distribution=dict(source_dist)
        )
        context_vector = features.to_vector()
        
        # Select strategy
        arm_index = self.bandit.select_arm(context_vector)
        strategy = self.bandit.get_strategy(arm_index)
        self.strategy_counts[strategy.value] += 1
        
        # Create dummy query for prioritizer
        query = Query(
            text=query_text,
            query_type=query_type_enum,
            complexity=query_complexity,
            user_id="user",
            embedding=np.zeros(64),
            timestamp=datetime.now().timestamp()
        )
        
        # Apply strategy
        selected = ContextPrioritizer.select_items(
            context_items, strategy, self.token_budget, query
        )
        
        # Convert back to dict format
        selected_dicts = [
            {
                "id": item.id,
                "content": item.content,
                "source": item.source.value,
                "token_count": item.token_count,
            }
            for item in selected
        ]
        
        metadata = {
            "strategy": strategy.value,
            "items_selected": len(selected),
            "tokens_used": sum(i.token_count for i in selected),
            "context_vector": context_vector.tolist(),
            "arm_index": arm_index,
        }
        
        return selected_dicts, strategy.value, metadata
    
    def record_feedback(
        self,
        metadata: Dict,
        response_quality: float,
        task_success: bool = True
    ):
        """
        Record feedback after LLM response to update the bandit.
        
        Args:
            metadata: The metadata returned from prioritize_context
            response_quality: 0-1, quality of the LLM response
            task_success: Whether the task was completed successfully
        """
        # Compute reward
        efficiency = 1.0 - (metadata["tokens_used"] / self.token_budget) * 0.1
        reward = 0.7 * response_quality + 0.2 * float(task_success) + 0.1 * efficiency
        reward = max(0, min(1, reward))
        
        # Update bandit
        context_vector = np.array(metadata["context_vector"])
        arm_index = metadata["arm_index"]
        self.bandit.update(context_vector, arm_index, reward)
        
        # Track metrics
        self.total_reward += reward
        self.recent_rewards.append(reward)
        if len(self.recent_rewards) > 100:
            self.recent_rewards.pop(0)
    
    def _convert_items(self, items: List[Dict]) -> List[ContextItem]:
        """Convert dict items to ContextItem objects"""
        converted = []
        for i, item in enumerate(items):
            source = item.get("source", "retrieved_docs")
            try:
                source_enum = ContextSource(source)
            except ValueError:
                source_enum = ContextSource.RETRIEVED_DOCS
                
            converted.append(ContextItem(
                id=item.get("id", f"item_{i}"),
                content=item.get("content", ""),
                source=source_enum,
                token_count=item.get("token_count", 100),
                timestamp=item.get("timestamp", datetime.now().timestamp()),
                relevance_score=item.get("relevance_score", 0.5),
                topic_embedding=np.random.randn(32),
                user_specific=item.get("user_specific", False)
            ))
        return converted
    
    def get_stats(self) -> Dict:
        """Get current optimizer statistics"""
        return {
            "total_requests": self.request_count,
            "avg_reward": self.total_reward / max(1, self.request_count),
            "recent_avg_reward": np.mean(self.recent_rewards) if self.recent_rewards else 0,
            "strategy_distribution": dict(self.strategy_counts),
            "algorithm": self.algorithm,
        }
    
    def save_model(self, path: str):
        """Save the learned bandit model"""
        with open(path, 'wb') as f:
            pickle.dump({
                'bandit': self.bandit,
                'stats': self.get_stats(),
            }, f)
        print(f"Model saved to {path}")
    
    def load_model(self, path: str):
        """Load a previously saved model"""
        with open(path, 'rb') as f:
            data = pickle.load(f)
            self.bandit = data['bandit']
        print(f"Model loaded from {path}")


print("✅ ProductionContextOptimizer defined!")
print("   Ready for integration with real agentic systems")

In [ ]:
# Demo: Using the Production Optimizer
print("=" * 70)
print("PRODUCTION OPTIMIZER DEMO")
print("=" * 70)

# Create optimizer
optimizer = ProductionContextOptimizer(
    algorithm="thompson_sampling",
    token_budget=4000
)

# Simulate some context items (in practice, these come from your RAG system)
sample_items = [
    {"id": "doc_1", "content": "API documentation for authentication...", 
     "source": "retrieved_docs", "token_count": 350, "relevance_score": 0.9},
    {"id": "doc_2", "content": "User guide for setup...", 
     "source": "retrieved_docs", "token_count": 280, "relevance_score": 0.7},
    {"id": "history_1", "content": "Previous conversation about login issues...", 
     "source": "conversation_history", "token_count": 420, "relevance_score": 0.85},
    {"id": "user_pref", "content": "User preferences and settings...", 
     "source": "user_profile", "token_count": 150, "relevance_score": 0.5},
    {"id": "system_1", "content": "Current system status: all services healthy...", 
     "source": "system_state", "token_count": 200, "relevance_score": 0.4},
    {"id": "doc_3", "content": "Troubleshooting guide for common errors...", 
     "source": "retrieved_docs", "token_count": 500, "relevance_score": 0.95},
    {"id": "api_1", "content": "Live API response with latest data...", 
     "source": "external_api", "token_count": 300, "relevance_score": 0.6},
    {"id": "cache_1", "content": "Cached response from similar query...", 
     "source": "cached_results", "token_count": 250, "relevance_score": 0.55},
]

# Process a query
selected, strategy, metadata = optimizer.prioritize_context(
    query_text="How do I fix the authentication error I'm seeing?",
    query_type="troubleshooting",
    query_complexity=0.6,
    available_items=sample_items,
    user_history_length=15
)

print(f"\n📝 Query: 'How do I fix the authentication error I'm seeing?'")
print(f"📊 Query type: troubleshooting")
print(f"\n🎯 Selected Strategy: {strategy}")
print(f"📦 Items selected: {metadata['items_selected']}")
print(f"🔢 Tokens used: {metadata['tokens_used']} / 4000")

print("\n📋 Selected context items:")
for item in selected:
    print(f"   - [{item['source']}] {item['id']}: {item['token_count']} tokens")

# Simulate feedback (in practice, this comes from user rating or task completion)
optimizer.record_feedback(metadata, response_quality=0.85, task_success=True)

print(f"\n📈 Optimizer stats after 1 interaction:")
stats = optimizer.get_stats()
for k, v in stats.items():
    print(f"   {k}: {v}")

## 9. Summary and Key Takeaways

### Why Contextual Bandits for Context Prioritization?

| Aspect | Contextual Bandits | Deep RL (PPO/DQN) |
|--------|-------------------|-------------------|
| **Sample Efficiency** | ✅ Learns quickly | ❌ Needs many samples |
| **Implementation** | ✅ Simple | ❌ Complex |
| **Interpretability** | ✅ Clear strategy selection | ❌ Black box |
| **Cold Start** | ✅ Works with heuristics | ❌ Requires pretraining |
| **Real-time Updates** | ✅ Online learning | ❌ Batch updates |

### Recommended Approach

1. **Start with Thompson Sampling** - Best balance of exploration/exploitation
2. **Use meaningful features** - Query type, complexity, source distribution
3. **Design good strategies** - Each arm should be a distinct prioritization approach
4. **Collect quality feedback** - User ratings, task success, or automated metrics

### Integration Checklist

- [ ] Define your context sources (docs, history, user profile, etc.)
- [ ] Implement feature extraction for your queries
- [ ] Create prioritization strategies tailored to your use case
- [ ] Set up feedback collection (explicit ratings or implicit signals)
- [ ] Deploy with A/B testing against baseline
- [ ] Monitor and iterate

### Next Steps

For more complex scenarios, consider:
- **Hierarchical bandits** for multi-level decisions
- **Neural contextual bandits** for richer feature representations
- **Full RL (DQN/PPO)** for sequential selection with complex dependencies

In [ ]:
# Final summary
print("=" * 70)
print("NOTEBOOK COMPLETE!")
print("=" * 70)
print("""
This notebook demonstrated:

📚 CONCEPTS
   • Context window prioritization as a bandit problem
   • Mapping prioritization strategies to bandit arms
   • Feature engineering for query context

🔧 IMPLEMENTATIONS
   • Epsilon-Greedy with linear models
   • Thompson Sampling (Bayesian approach)
   • LinUCB (optimistic exploration)
   • 6 different prioritization strategies

📊 RESULTS
   • Thompson Sampling typically performs best
   • Learns query-type specific policies
   • Low regret compared to random baseline

🚀 PRODUCTION
   • ProductionContextOptimizer class ready for integration
   • Save/load trained models
   • Real-time feedback and learning

Next: Try integrating with your RAG pipeline!
""")